In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# import data
data = pd.read_csv("")

In [ ]:
# pd.to_datetime() 是 Pandas 库中的一个函数，用于将参数转换为日期时间格式
data.index = pd.to_datetime(data.index) 

In [ ]:
# 如何将一个数据集分割成训练集、测试集和保留集（holdout set），以及如何为使用统计模型进行回归分析准备这些数据集

# 计算分割点，用于将数据集分为三个部分
split1 = int(len(data) * .6) # split1 是数据长度的60%，标志着训练集（train set）的终点和测试集（test set）的起点
split2 = int(len(data) * .9) # split2 是数据长度的90%，标志着测试集的终点和保留集的起点

# 根据计算出的分割点，将数据分割成三个部分
data_in = data[:split1] # data_in 作为训练集
data_out = data[split1:split2] # data_out 作为测试集
data_hold = data[split2:] # data_hold 作为保留集

# 从训练集中分离出特征（X_in）和目标变量（y_in）
X_in = data_in.iloc[:,1:] # 包含了除了第一列之外的所有列（假设第一列是目标变量）
y_in = data_in.iloc[:,0] # 只包含第一列

# 测试集分离出特征和目标变量
X_out = data_out.iloc[:,1:]
y_out = data_out.iloc[:,0]

# 保留集分离出特征和目标变量
X_hold = data_hold.iloc[:,1:]
y_hold = data_hold.iloc[:,0]

# 通过添加一个常数列（通常作为截距项）来为每个特征集准备数据
X_in = sm.add_constant(X_in)
X_out = sm.add_constant(X_out)
X_hold = sm.add_constant(X_hold)

In [ ]:
# 使用 statsmodels 库构建一个线性回归模型
model1 = sm.OLS(y_in, X_in).fit()

In [ ]:
# matplotlib 库绘制图形
plt.figure(figsize = (12,5))

plt.plot()
plt.plot(y_out)
plt.plot(model1.predict(X_in))
plt.plot(model1.predict(X_out))

plt.ylabel("Fed Funds")
plt.xlabel("Time")
plt.title("Visualizing Model Accuracy")
plt.legend([])
plt.grid()
plt.show()

In [ ]:
# mean_squared_error 函数计算两个序列之间的均方误差

# 计算训练集的实际值 (y_in) 与模型在训练集上的预测值 (model1.predict(X_in)) 之间的 MSE
in_mse_1 = mean_squared_error(y_in, model1.predict(X_in))

# 计算测试集的实际值 (y_out) 与模型在测试集上的预测值 (model1.predict(X_out)) 之间的 MSE
out_mse_1 = mean_squared_error(y_out, model1.predict(X_out))

# 通过在测试集上计算 MSE，可以评估模型对新数据的泛化能力。
# 理想情况下，训练集和测试集上的 MSE 应该都比较低且彼此接近，这表明模型既能很好地拟合训练数据，也能很好地泛化到未见过的数据。
# 通过比较 in_mse_1（训练集 MSE）和 out_mse_1（测试集 MSE），我们可以对模型的性能和其在未知数据上的表现有一个直观的了解。
# 如果测试集的 MSE 明显高于训练集的 MSE，这可能是过拟合的一个迹象，意味着模型过于复杂，学习了训练数据中的噪声而非潜在的数据模式。

In [ ]:
# 通过增加多项式特征的度数来探索和比较不同复杂度的模型对于给定数据的拟合程度。
# 它使用一个loop来逐步增加多项式特征的度数，并对每个度数的模型进行训练和评估。
from sklearn.preprocessing import PolynomialFeatures

max_degrees = 3

for degrees in range(1,max_degrees + 1):
    print("Degrees:", degrees)
    # PolynomialFeatures 是 sklearn.preprocessing 模块的一个类，用于生成一个新的特征矩阵，
    # 其中包含了原始特征的所有多项式组合，直到指定的度数 degrees。这使得模型能够探索特征之间的非线性关系。
    poly = PolynomialFeatures(degree = degrees)
    
    # 首先对多项式特征生成器进行拟合，以学习数据的特征，然后将这些特征转换为指定度数的多项式特征
    X_in_poly = poly.fit_transform(X_in) # X_in_poly 包含了训练数据 X_in 的多项式扩展
    X_out_poly = poly.transform(X_out) # 使用与训练数据相同的多项式转换来转换测试数据 X_out，得到 X_out_poly
    
    model1 = sm.OLS(y_in, X_in_poly).fit()
    
    # 绘图，显示了训练集和测试集中实际值 (y_in 和 y_out) 以及模型预测值 (in_preds 和 out_preds)。
    # 直观地展示模型在不同多项式度数下对数据的拟合程度
    
    plt.figure(figsize = (12,5))
    
    # 预测值是将多项式转换后的特征传递给模型得到的
    in_preds = model1.predict(X_in_poly)
    in_preds = pd.DataFrame(in_preds, index = y_in.index)
    
    out_preds = model1.predict(X_out_poly)
    out_preds = pd.DataFrame(out_preds, index = y_out.index)
    
    plt.plot(y_in)
    plt.plot(y_out)
    plt.plot(in_preds)
    plt.plot(out_preds)

    plt.ylabel("Fed Funds")
    plt.xlabel("Time")
    plt.title("Visualizing Model Accuracy")
    plt.legend([])
    plt.grid()
    plt.show()
    
    # 使用 mean_squared_error 函数计算模型在训练集 (X_in_poly) 和测试集 (X_out_poly) 上的均方误差
    # 提供了量化模型性能的另一种方式，可以用来比较不同复杂度模型的过拟合或欠拟合情况。
    in_mse_1 = mean_squared_error(y_in, model1.predict(X_in_poly))
    out_mse_1 = mean_squared_error(y_out, model1.predict(X_out_poly))
    print("Insample MSE : ", in_mse_1)
    print("Outsample MSE : ", out_mse_1)

# 通过探索不同的多项式度数来寻找模型的最佳复杂度，这样可以最好地拟合数据而不产生过拟合。
# 随着多项式度数的增加，模型能够捕获更复杂的非线性关系，但同时也有过度拟合数据的风险，特别是当度数很高时。
# 通过比较不同模型的训练和测试误差，可以找到最适合数据的模型复杂度。